# Apprentissage supervisé
### Approche expérimentale et classifieur naïf bayesien pour la détection de polarité

CSI4506 Intelligence Artificielle  
Automne 2018  
Caroline Barrière

***


La tâche de classification que nous attaquons dans ce notebook est la **détection de polarité** (*polarity detection*), qui est une des tâches faisant partie du champ de recherche de l'analyse d'opinion (*Opinion Mining*), très populaire de nos jours.  Plusieurs compagnies ont envie de savoir ce que les gens pensent d'elles.  Des commentaires (*reviews*) peuvent être faits sur des films, des hotels, des restaurants, des services à la clientèle, etc. 

Ce notebook insiste sur l'utilisation d'une bonne **approche experimentale** pour l'apprentissage machine dans laquelle nous regarderons les notions d'ensemble d'entraînement, d'ensemble test, d'évaluation, de biais, etc.  Aussi, le notebook montre l'importance de l'évaluation comparative.  Avant de dire qu'une méthode est bonne ou non... il est important de la comparer à d'autres méthodes.  Une bonne approche expérimentale inclut le développement d'un algorithme basique (*baseline*) auquel les méthodes plus avancées pourront se comparer.  

Ce notebook fait usage d'un *package Python* très utilisé en apprentissage machine, soit **scikit-learn** (http://scikit-learn.org/stable/).  Ce package contient des algorithmes pré-codés d'apprentissage machine.  Pour utiliser ce package, vous devez le télécharger et l'installer.  Pour ce faire, à l'invite de commande, tapper *pip install sklearn*.  

Ce notebook test l'implémentation du Naive Bayes proposé par scikit-learn.  Nous testerons probablement d'autres algorithmes pré-codés dans ce package dans des notebooks futurs.

***


***DEVOIR***:  
Parcourir le notebook, en exécutant chaque cellule, une à une.  
Pour chaque **(TO DO)**, effectuer les tâches demandées.  
Quand vous avez terminé, signez et soumettez votre notebook.

***


**1. Détection de polarité**  

La détection de polarité se base en général sur 2 classes: Positif et Négatif.  Ceci est différent de l'analyse de sentiment par exemple où nous aurions plusieurs classes telles joyeux, triste, anxieus, fâché, etc.  La détection de polarité est aussi une tâche plus restreinte que l'attribution de cote (e.g. 0...5 de mauvais à très bon). Ainsi, la tâche d'analyse de polarité vise à attribuer automatiquement à un commentaire non-vu, une valeur *positive* ou *negative*.

**2. Domaine d'application:  Commentaires sur les films**  

La détection de polarité peut se faire sur n'importe quels types de commentaires.  Ici, j'ai choisi le domaine des films pour tester l'approche expérimentale que nous développerons.  Dans ce domaine d'application, nous développons d'abord un ***ensemble de test*** qui nous servira à mesurer les valeurs prédictives de nos algorithmes.  Nous ne **DEVONS PAS** utiliser cet ensemble test pour l'apprentissage de modèles prédictifs (que nous ferons plus tard).

In [1]:
# let's establish a few test sentences. 
# For each sentence, we have a corresponding tag: "Neg" or "Pos"

test_reviews = ["Can't believe I wasted my time on this", 
                "Really awful",
                "Actors were good",
                "So boring",
                "Stayed at the edge of my seat",
                "I never like any movie",
                "Argghhhh I hated it"]

test_tags = ["Neg", "Neg", "Pos", "Neg", "Pos", "Neg", "Neg"]

**3. Biais:  Ressources disponibles**  

Pour la détection de polarité, des chercheurs ont établi des listes de mots positifs et négatifs.  Les listes que nous utiliserons dans ce notebook peuvent être téléchargées [ici](https://www.cs.uic.edu/~liub/FBS/sentiment-analysis.html) (un site sur la fouille d'opinion du chercheur reconnu Bing Lu) et sauvegardée localement.

J'ai inclu les fichiers *positive-words.txt* et *negative-words.txt* dans le module Jupyter Notebook dans Brightspace.  Assurez-vous de placer ces fichiers dans le même répertoire que votre notebook, ou encore, de modifier le code ci-bas pour inclure le chemin vers le fichier à son ouverture.  

Tel que nous avons discuté en classe, l'utilisation de ressources externes est un peu un *biais* que nous introduisons dans notre étude du problème, car cette information ne provient pas de nos données à analyser.  Mais dans le cas présent... les listes ont elles-mêmes été compilées par d'autres chercheurs lors d'autres analyses...

In [2]:
# Read the positive words

with open("positive-words.txt") as f:
    posWords = f.readlines()
posWords = [p[0:len(p)-1] for p in posWords if p[0].isalpha()] 

# print the first 50 words
print(posWords[:50])

['a+', 'abound', 'abounds', 'abundance', 'abundant', 'accessable', 'accessible', 'acclaim', 'acclaimed', 'acclamation', 'accolade', 'accolades', 'accommodative', 'accomodative', 'accomplish', 'accomplished', 'accomplishment', 'accomplishments', 'accurate', 'accurately', 'achievable', 'achievement', 'achievements', 'achievible', 'acumen', 'adaptable', 'adaptive', 'adequate', 'adjustable', 'admirable', 'admirably', 'admiration', 'admire', 'admirer', 'admiring', 'admiringly', 'adorable', 'adore', 'adored', 'adorer', 'adoring', 'adoringly', 'adroit', 'adroitly', 'adulate', 'adulation', 'adulatory', 'advanced', 'advantage', 'advantageous']


In [3]:
# Read the negative words

with open("negative-words.txt") as f:
    negWords = f.readlines()
negWords = [p[0:len(p)-1] for p in negWords if p[0].isalpha()] 

print(negWords[:50])

['abnormal', 'abolish', 'abominable', 'abominably', 'abominate', 'abomination', 'abort', 'aborted', 'aborts', 'abrade', 'abrasive', 'abrupt', 'abruptly', 'abscond', 'absence', 'absent-minded', 'absentee', 'absurd', 'absurdity', 'absurdly', 'absurdness', 'abuse', 'abused', 'abuses', 'abusive', 'abysmal', 'abysmally', 'abyss', 'accidental', 'accost', 'accursed', 'accusation', 'accusations', 'accuse', 'accuses', 'accusing', 'accusingly', 'acerbate', 'acerbic', 'acerbically', 'ache', 'ached', 'aches', 'achey', 'aching', 'acrid', 'acridly', 'acridness', 'acrimonious', 'acrimoniously']


**4. Approche basique (*Baseline approach*)**  

Avant d'évaluer les performances d'une approche supervisée, nous débutons par une approche très simple.  C'est toujours bien de débuter simple, et ainsi de pouvoir mesurer les gains qu'apporteront (ou non) nos algorithmes plus complexes.

L'*approche basique* que nous utiliserons comparera tout simplement le nombre de mots positifs et négatifs des commentaires à analyser et utilisera la classe du nombre maximum.  Cette approache N'APPREND PAS. Mais elle applique tout de même un *raisonnement simple* au moment du test.  Vous seriez surpris de savoir combien de *start-up AI* s'attaquant à l'analyse d'opinion, utilisent ce genre de méthode simple. 

In [4]:
# first let's define methods to count positive and negative words

def countPos(text):
    count = 0
    for t in text.split():
        if t in posWords:
            count += 1
    return count

def countNeg(text):
    count = 0
    for t in text.split():
        if t in negWords:
            count += 1
    return count

In [5]:
# simple counting algorithm as baseline approach to polarity detection
def baselinePolarity(review):
    numPos = countPos(review)
    numNeg = countNeg(review)
    if numPos > numNeg:
        return "Pos"   
    else:
        return "Neg"   

In [6]:
# Test the baseline method
print(baselinePolarity("This was a really good movie"))

Pos


**5. Évaluation**  
Nous avons vu en classe qu'il existe de multiples façons d'évaluer les prédictions d'un algorithme.  Dans le cas d'une tâche de classification comme nous avons ici, une méthode d'évaluation peut être de calculer le *nombre de mauvaises assignations*.  

Pour tester notre approche basique, calculons ci-bas le nombre de mauvaises assignations sur notre ensemble de test.

In [7]:
# Let's establish the polarity for each review

nbWrong = 0
for i in range(len(test_reviews)):
    polarity = baselinePolarity(test_reviews[i])
    print(test_reviews[i] + " -- " + polarity)
    if (polarity != test_tags[i]):
        nbWrong += 1

print('\nThere are %s wrong assignments' %nbWrong)    

Can't believe I wasted my time on this -- Neg
Really awful -- Neg
Actors were good -- Pos
So boring -- Neg
Stayed at the edge of my seat -- Neg
I never like any movie -- Pos
Argghhhh I hated it -- Neg

There are 2 wrong assignments


**(TO DO - Q1)** Créer un petit ensemble de test de 6 commentaires de film de votre choix.  Ensuite, appliquer l'algorithme basique sur votre nouvel ensemble de test et calculer le nombre de mauvaises assignations.  Pour ce calcul, vous pouvez copier-coller le code ci-haut et le modifier légèrement pour l'appliquer sur votre ensemble test.

In [8]:
# new test set
my_reviews = ["Absolutely awesome", "Terrible", "Excellent filmography", "Meh",
              "Breath-taking", "Piece of garbage", "My new favourite"]
my_tags = ["Pos", "Neg", "Pos", "Neg", "Pos", "Neg", "Pos"]

# test baseline polarity algorithm
nbWrong = 0
for i in range(len(my_reviews)):
    polarity = baselinePolarity(my_reviews[i])
    print(my_reviews[i] + " -- " + polarity)
    if (polarity != my_tags[i]):
        nbWrong += 1

print('\nThere are %s wrong assignments' %nbWrong)    

Absolutely awesome -- Pos
Terrible -- Neg
Excellent filmography -- Neg
Meh -- Neg
Breath-taking -- Neg
Piece of garbage -- Neg
My new favourite -- Neg

There are 3 wrong assignments


#### 6. Supervised learning method

Nous allons maintenant entraîner un modèle d'apprentissage supervisé pour la détection de polarité.  Si ce n'est pas déjà fait, vous devrez installer le package **sklearn**, utilisant *pip install sklearn*.  

***6.1 Ensemble d'entraînement***  

Pour effectuer un apprentissage supervisé, nous avons besoin d'un ensemble d'entraînement.  Cet ensemble devrait être ***différent*** mais tout de même ***représentatif*** de l'ensemble test.

Établissons un *ensemble d'entraînement* ci-bas.  D'habitude, l'ensemble d'entraînement devrait être aussi grand et varié que possible.  Les ensembles d'entraînements sont très précieux, car ils demandent souvent beaucoup d'effort humain pour les obtenir.

In [9]:
# small training set... normally we require hundreds of sentences

train_reviews = ["this movie was stupid, so stupid",
                  "I hated this movie",
                  "I loved it",
                  "What a waste of time",
                  "Amazing!",
                  "What a pity",
                  "Very good movie indeed.  Glad I saw it."]

train_tags = ["Neg", "Neg", "Pos", "Neg", "Pos", "Neg", "Pos"]

***6.2 Pre-traitement de l'information*** 

Le package *scikit-learn* est assez particulier quand au format des données pour débuter l'entraînement de modèles.  Ainsi, nous devrons effectuer quelques étapes de pré-traitement sur les données.  Heureusement, *scikit-learn* fournit des méthodes pour faciliter le pré-traitement.  

Comme pré-traitement, nous devons transformer chaque phrase en une liste de mots, et chaque mot aura un index associé dans un dictionnaire.  Les clés du dictionnaire python sont les mots, et les valeurs sont les index.

In [10]:
from sklearn.feature_extraction.text import CountVectorizer

# The CountVectorizer builds a dictionary of all words (count_vect.vocabulary_), 
# and generates a matrix (train_counts), to represent each sentence
# as a set of indices into the dictionary. The words in the dictionary are the words found in train_reviews.

count_vect = CountVectorizer()
train_counts = count_vect.fit_transform(train_reviews)

ModuleNotFoundError: No module named 'sklearn'

Pour comprendre le code ci-haut, affichons le vocabulaire (tous les mots) extraits des phrases d'entraînement.

In [ ]:
# print the vocabulary (dictionary of words)
print(count_vect.vocabulary_)

Nous pouvons comprendre l'affichage ci-haut comme: 

'saw':10  veut dire que le mot 'saw' est assigné à l'index 10  
'time':14 veut dire que le mot 'time' est assigné à l'index 14 

Maintenant affichons le contenu de *train_counts*.  

In [ ]:
# print the content of the training examples in terms of frequency of words (each word represented by its index)
print(train_counts)

On peut comprendre chaque ligne affichée ci-haut comme: 

(0, 11) 1  -- phrase 0 (de train_reviews) contient 1 fois le mot 11 (11 est l'index du mot 'so' dans count_vect.vocabulary)  
(0, 12) 2  -- phrase 0 (de train_reviews) contient 2 fois le mot 12 (12 est l'index du mot 'stupid' dans count_vect.vocabulary)

C'est donc que chaque phrase de l'ensemble d'entrainement est représentée comme un BOW (bag of words), et donc chaque phrase devient une liste d'index, chaque index étant associé à un mot.

***6.3 Apprentissage naïf bayesien***

Avec l'ensemble d'entraînement pré-traité, nous sommes prêts pour utiliser l'algorithme Naive Bayes de scikit-learn.  Cet algorithm nécessite que les données d'apprentissage soient représentées en terme de *train counts* (nombre d'occurrence de chaque attribut), d'où le pré-traitement que nous venons d'effectuer.

L'étape d'apprentissage du modèle se fait par la méthode *fit*, tel que vous voyez ci-bas.  Mais vous savez ce qui se cache sous le *fit*.  En effet, la méthode fait un calcul des probabilité a priori des hypothèses (Neg, Pos) et des probabilités a posteriori des mots conditionnellement aux classes (e.g. P(stupid|Pos) or P(stupid|Neg)).  Toutes ces probabilités seront nécessaires lors de l'étape de test qui utilisera le théorème de Bayes.

**(TO-DO - Q2)** Quelles sont les probabilités a priori des classes Pos et Neg selon l'ensemble d'entraînement ci-haut?  (cette question ne demande pas de coder quoi que ce soit, juste de le faire à la main).

Answers

P(Pos) = 3/7

P(Neg) = 4/7

In [ ]:
# The learning step of the naive bayes algorithm, the "fit" is the training
from sklearn.naive_bayes import MultinomialNB

# Training the model
clf = MultinomialNB().fit(train_counts, train_tags)   


***6.4 Evaluation***

Évaluons d'abord le modèle appris (clf) sur l'ensemble d'entraînement.  Pour appliquer ce modèle pour la classification (*prediction*), nous utilisons la méthode *predict* ci-bas. 

In [ ]:
# Testing on training set
predicted = clf.predict(train_counts)

# Afficher les prédictions
for doc, category in zip(train_reviews, predicted):   # zip allows to go through two lists simultaneously
    print('%r => %s' % (doc, category))

C'est sans surprise que sur l'ensemble d'entraînement, le modèle obtient 100%...  Mais nous devrions évaluer le modèle sur un ensemble **TEST**.  Nous avons 2 ensembles tests déjà construits.  Le premier est celui que j'avais créé ci-haut (*test_reviews*) et l'autre est celui que vous avez créé (*my_review*).

**(TO_DO - Q3)** Évaluer le modèle entraîné sur les deux ensembles tests.  Avant chaque test, vous devez pré-traiter l'ensemble test avec l'étape de prétraitement vue ci-haut, pour rendre vos ensembles tests compatibles avec ce que l'apprenant (clf) a besoin.

In [ ]:
# Pre-process test set test_reviews
test_counts = count_vect.transform(test_reviews)
# predict the results
test_predicted = MultinomialNB().fit(test_counts, test_tags).predict(test_counts)
# print the results
for doc, category in zip(test_reviews, test_predicted):
    print('%r => %s' % (doc, category))

print()

# Pre-process the test set my_reviews
my_counts = count_vect.transform(my_reviews)
# predict the results
my_predicted = MultinomialNB().fit(my_counts, my_tags).predict(my_counts)
# print the results
for doc, category in zip(my_reviews, my_predicted):
    print('%r => %s' % (doc, category))

**(TO_DO - Q4)** Une **mesure d'évaluation**, reliée au *nombre de mauvaises assignations* est la mesure de **Rappel**  Le rappel, mesuré sur une classe, est le nombre de bonnes assignations obtenues sur cette classe divisé par le nombre d'exemples appartenant à cette classes dans l'ensemble test.  Par exemple, si l'ensemble test contient 5 phrases positives et que l'algorithme est trouve 2, alors son rappel sur la classe Pos est de 2/5.

Écrivez une petite méthode ci-bas pour calculer le rappel.  La méthode devrait avoir 3 paramètres, le premier paramètre contenant la liste des tags attendus (e.g. (Pos, Neg, Pos)), le deuxième paramètre contenant la liste des tags prédits (e.g (Pos, Pos, Neg)), le troisième contient la classe (e.g. Pos).  La méthode devra retourner le rappel (e.g. 50%).

In [ ]:
# Number wrong
def rappel(goodTags, predictions, classe):
    nbWrong = 0
    count = 0
    for first, second in zip(goodTags, predictions):
        if first == classe:
            count+=1
            if first != second:
                nbWrong+=1
    return str(nbWrong) + '/' + str(count);

**(TO DO - Q5)** Utiliser la méthode de rappel ci-haut pour calculer le rappel des 2 ensembles tests.  Afficher les résultats.

In [ ]:
# rappel
print(rappel(test_tags, test_predicted, 'Pos'))
print(rappel(test_tags, test_predicted, 'Neg'))
print(rappel(my_tags, my_predicted, 'Pos'))
print(rappel(my_tags, my_predicted, 'Neg'))

#### 7. Discussion

**(TO DO - Q6)** Est-ce que la méthode naive bayesienne fait mieux que l'approche basique?  Présenter et discuter les résultats. Donner 2 suggestions de ce qui pourrait être fait pour aider l'algorithme naïf bayesien en lien avec avec notre petite expérimentation de détection de polarité.

Résultats comparatifs:
L'approche basique obtient 3 mauvais resultats sur 7 entrees dans mon ensemble test tandis que l'approche bayesienne en obtient 2. Sur l'ensemble test les deux approches obtiennent 2 mauvais resultats


Deux suggestions pour aider l'algo bayesien:  
1) tenir compte du nombre de mots positifs et negatifs comme fait l'approche basique

2) 

**(TO-DO - OPTIONAL)**  Nous avons utilisé uniquement 2 classes: Positif et Négatif.  Mais... il pourrait très bien y avoir une troisième classe Neutre, qui serait utilisée pour les commentaires du genre "I don't know" qui ne sont pas informatifs quand à leur polarité.   Vous pouvez reprendre l'approche expérimentale ci-haut pour faire des tests avec l'approche basique et avec le classifieur naïf bayesien pour 3 classes. Si vous faites ce travail optionnel, faites-le ci-bas, ne modifiez pas directement les cellules précédentes.

#### Signature

Je, Oliver Scott, declare que les réponses inscrites dans ce notebook sont les miennes.